## 01. data preprocess

In [1]:
import pandas as pd
import os

In [2]:
df_anno = pd.read_csv("../data/raw/flickr8k/ExpertAnnotations.txt", header=None, delimiter="\t")
df_anno = df_anno.rename(columns={0:"image", 1:"cand_caption"})
df_anno.head(20)

,image,cand_caption,2,3,4
0,1056338697_4f7d7ce270.jpg,2549968784_39bfbe44f9.jpg#2,1,1,1
1,1056338697_4f7d7ce270.jpg,2718495608_d8533e3ac5.jpg#2,1,1,2
2,1056338697_4f7d7ce270.jpg,3181701312_70a379ab6e.jpg#2,1,1,2
3,1056338697_4f7d7ce270.jpg,3207358897_bfa61fa3c6.jpg#2,1,2,2
4,1056338697_4f7d7ce270.jpg,3286822339_5535af6b93.jpg#2,1,1,2
5,1056338697_4f7d7ce270.jpg,3360930596_1e75164ce6.jpg#2,1,1,1
6,1056338697_4f7d7ce270.jpg,3545652636_0746537307.jpg#2,1,1,1
7,1056338697_4f7d7ce270.jpg,434792818_56375e203f.jpg#2,1,1,2
8,106490881_5a2dd9b7bd.jpg,1425069308_488e5fcf9d.jpg#2,1,1,1
9,106490881_5a2dd9b7bd.jpg,1714316707_8bbaa2a2ba.jpg#2,2,2,2


In [5]:
df_cap = pd.read_csv("../data/raw/flickr8k/Flickr8k.token.txt", header=None, delimiter="\t")
df_cap = df_cap.rename(columns={0:"image", 1:"cand_caption"})
df_cap.head(5)

,image,cand_caption
0,1000268201_693b08cb0e.jpg#0,A child in a pink dress is climbing up a set o...
1,1000268201_693b08cb0e.jpg#1,A girl going into a wooden building .
2,1000268201_693b08cb0e.jpg#2,A little girl climbing into a wooden playhouse .
3,1000268201_693b08cb0e.jpg#3,A little girl climbing the stairs to her playh...
4,1000268201_693b08cb0e.jpg#4,A little girl in a pink dress going into a woo...


In [6]:
# build look-up dictionary

with open("../data/raw/flickr8k/Flickr8k.token.txt") as f:
    cap = f.readlines()
    cap = [x.strip() for x in cap]
    
cap_dict = {}
for item in cap:
    _id    = item.split("\t")[0].split("#")[0]
    _index = item.split("\t")[0].split("#")[1]
    _text  = item.split("\t")[1]
    
    if _id in cap_dict:
        cap_dict[_id].append(_text)
        
    else:
        cap_dict[_id] = [_text]        

In [7]:
# image 
imgs = df_anno["image"]
base_path = "../data/raw/flickr8k/Flicker8k_Dataset/"

if os.path.isdir("../data/processed/") == False:
    os.mkdir("../data/processed/")
    
if os.path.isdir("../data/processed/flickr8k") == False:
    os.mkdir("../data/processed/flickr8k")
    

with open("../data/processed/flickr8k/imgs.txt", "w") as f:
    for item in imgs:
        f.write(base_path + item + "\n")

In [8]:
# cand caption
cand = df_anno["cand_caption"]

with open("../data/processed/flickr8k/cand.txt", "w") as f:
    for item in cand:
        _id    = item.split("#")[0]
        _index = int(item.split("#")[1])
        f.write( cap_dict[_id][_index] + "\n")

In [9]:
# gt caption
gt = df_anno["image"]

with open("../data/processed/flickr8k/gt.txt", "w") as f:
    for item in gt:
        for _text in cap_dict[item]:
            f.write( _text + "\n")

In [11]:
# annotations

anno_1 = df_anno[2]
anno_2 = df_anno[3]
anno_3 = df_anno[4]
anno_avg = (anno_1 + anno_2 + anno_3)/3

with open("../data/processed/flickr8k/annotations.txt", "w") as f:
    for i in range(len(anno_1)):
        f.write(str(anno_1[i]) + "," + str(anno_2[i]) + "," + str(anno_3[i]) +"\n")
        
with open("../data/processed/flickr8k/annotations_avg.txt", "w") as f:
    for i in range(len(anno_avg)):
        f.write(str(anno_avg[i]) + "\n")

In [9]:
# sanity check

In [10]:
# import pickle
# with open("../data/flickr8k/gt_caps.pkl", "rb") as f:
#     tmp_gt = pickle.load(f)
    
# with open("../data/flickr8k/cand_caps.pkl", "rb") as f:
#     tmp_cand = pickle.load(f)
    
# with open("../data/flickr8k/scores.pkl", "rb") as f:
#     tmp_score = pickle.load(f, encoding='latin1')
    
# with open("../data/processed/flickr8k/annotations_avg.txt") as f:
#     anno = f.readlines()
#     anno = [float(x.strip()) for x in anno]

In [11]:
# ori_index = 14
# print("ori_index: ", ori_index)
# print("\nori: ", "\n", tmp_cand[ori_index], "\n", tmp_gt[ori_index], "\n", tmp_score[ori_index])

# targe_index = cand_caps.index(tmp_cand[ori_index])
# print("\ntarge_index: ", targe_index)
# print("\ntarget : ", "\n", cand_caps[targe_index], "\n", gt_caps[targe_index*5:targe_index*5+5], "\n", anno[targe_index])

## 02-A compute score - feature extraction

In [9]:
IMAGE_PATH = "../data/processed/flickr8k/imgs.txt"         # input
# IMAGE_PATH = "../data/processed/flickr8k/imgs_rep_5.txt"         # input
IMAGE_FEATURE = "../data/processed/flickr8k/image_features.pkl"  # output

FEATURE_EXTRACTOR_BATCH_SIZE = 4
DETECTRON_MODEL  = "../data/detection/detectron_model.pth"
DETECTRON_CONFIG = "../data/detection/detectron_config.yaml"

In [10]:
import sys
sys.path.append("../vilbert")
from extract_features_custom import FeatureExtractor

feature_extractor = FeatureExtractor(model_file    = DETECTRON_MODEL,
                                     config_file   = DETECTRON_CONFIG,
                                     batch_size    = FEATURE_EXTRACTOR_BATCH_SIZE,
)


====================== EVALUATION CONFIGURATION ======================
Summary
----------------------------------------------------------------------
 - model_file                         : ../data/detection/detectron_model.pth
 - config_file                        : ../data/detection/detectron_config.yaml
 - batch_size                         : 4
 - num_features                       : 100
 - output_folder                      : 
 - image_dir                          : 
 - feature_name                       : fc6
 - confidence_threshold               : 0
 - background                         : False
 - partition                          : 0


In [11]:
with open(IMAGE_PATH) as f:
    image_path = f.readlines()

image_path = [x.strip() for x in image_path]
img_features = feature_extractor.extract_features_direct(image_path)

assert len(image_path) == len(img_features)

1456it [1:39:17,  3.52s/it]


In [12]:
import pickle
with open(IMAGE_FEATURE, "wb") as f:
    pickle.dump(img_features, f)

## 02-B compute score

In [13]:
IMAGE_FEATURE = "../data/processed/flickr8k/image_features.pkl"  # output from step 01
GENERATED_CAPTION = "../data/processed/flickr8k/cand.txt"
GT_CAPTION = "../data/processed/flickr8k/gt.txt"

VS_BATCH_SIZE = 400

In [14]:
import pandas as pd
import sys
sys.path.append("../vilbert")
import pickle
from compute_vilbertscore_custom import VilbertScore
vs = VilbertScore(batch_size=VS_BATCH_SIZE)

In [15]:
with open(IMAGE_FEATURE, "rb") as f:
    imgs = pickle.load(f)
            
with open(GENERATED_CAPTION, "r") as f:
    cand_caps = f.readlines()
    cand_caps = [x.strip() for x in cand_caps]

with open(GT_CAPTION, "r") as f:
    gt_caps = f.readlines()
    gt_caps = [x.strip() for x in gt_caps]
    
print(len(imgs), len(cand_caps), len(gt_caps))
num_itr = int(len(gt_caps) / len(cand_caps))
print("num_itr: ", num_itr)

5822 5822 29110
num_itr:  5


In [16]:
# multi gt case
for i in range(num_itr):
    
    # subset of gt caption
    sub_gt_cap = []
    for index, cap in enumerate(gt_caps):
        if index % num_itr == i:
            sub_gt_cap.append(cap)    
    
    # load dataset to compute
    vs.loaddata(list_image_feature = imgs,
                list_gen_caption = cand_caps,
                list_gt_caption = sub_gt_cap,
                max_len=50
               )
    
    precision, recall, f1 = vs.compute()
    
    if i==0:
        df = pd.DataFrame(data=[precision, recall, f1]).T
    else:
        df_tmp = pd.DataFrame(data=[precision, recall, f1]).T
        df = pd.concat([df, df_tmp])
        
df = df.rename(columns={0:"precision", 1:"recall", 2:"f1"})

  0%|          | 0/15 [00:00<?, ?it/s]

target data
Generated Captions (2 samples):  ['A young child is wearing blue goggles and sitting in a float in a pool .', 'A girl wearing a yellow shirt and sunglasses smiles .']
Ground truth Captions (2 samples):  ['A blond woman in a blue shirt appears to wait for a ride .', 'A blond woman in a blue shirt appears to wait for a ride .']


  0%|          | 0/15 [00:00<?, ?it/s]

target data
Generated Captions (2 samples):  ['A young child is wearing blue goggles and sitting in a float in a pool .', 'A girl wearing a yellow shirt and sunglasses smiles .']
Ground truth Captions (2 samples):  ['A blond woman is on the street hailing a taxi .', 'A blond woman is on the street hailing a taxi .']


  0%|          | 0/15 [00:00<?, ?it/s]

...

  0%|          | 0/15 [00:00<?, ?it/s]

target data
Generated Captions (2 samples):  ['A young child is wearing blue goggles and sitting in a float in a pool .', 'A girl wearing a yellow shirt and sunglasses smiles .']
Ground truth Captions (2 samples):  ['A woman with blonde hair wearing a blue tube top is waving on the side of the street .', 'A woman with blonde hair wearing a blue tube top is waving on the side of the street .']


  0%|          | 0/15 [00:00<?, ?it/s]

target data
Generated Captions (2 samples):  ['A young child is wearing blue goggles and sitting in a float in a pool .', 'A girl wearing a yellow shirt and sunglasses smiles .']
Ground truth Captions (2 samples):  ['The woman in the blue dress is holding out her arm at oncoming traffic .', 'The woman in the blue dress is holding out her arm at oncoming traffic .']


100%|██████████| 15/15 [00:54<00:00,  3.25s/it]


In [17]:
df

,precision,recall,f1
0,0.761258,0.760525,0.760891
1,0.759970,0.771912,0.765894
2,0.766312,0.787093,0.776564
3,0.797615,0.812368,0.804924
4,0.703377,0.662452,0.682301
...,...,...,...
5817,0.654174,0.654434,0.654304
5818,0.762904,0.768542,0.765713
5819,0.685805,0.707417,0.696443
5820,0.654777,0.684836,0.669470


### analysis

In [24]:
import numpy as np

In [27]:
scores_precision = []
for index in range( len(cand_caps) ):
    scores_precision.append( np.sum(df["precision"][index])/5 )
    
scores_recall = []
for index in range( len(cand_caps) ):
    scores_recall.append( np.sum(df["recall"][index])/5 )
    
scores_f1 = []
for index in range( len(cand_caps) ):
    scores_f1.append( np.sum(df["f1"][index])/5 )

In [36]:
df_rst = pd.DataFrame([scores_precision, scores_recall, scores_f1]).T
df_rst = df_rst.rename(columns={0:"precision", 1:"recall", 2:"f1"})

In [40]:
df_rst.to_csv("../data/processed/flickr8k/vilbertscore.csv", index=False)